# Estudo do Problema da Mochila
### Autor: José Brito

O dataset utilizado neste estudo foi criado por mim como parte de um cenário de jogo de RPG, onde o objetivo é decidir quais itens um personagem deve carregar em sua mochila (que suporta `10kg`)para maximizar o valor total dos objetos, sem ultrapassar o limite de peso. Cada item representa um tesouro ou item raro que pode ser encontrado durante a aventura, e o desafio é escolher quais itens levar, considerando o peso e o valor de cada um.

As colunas do dataset são as seguintes:
- **Item**: Nome do objeto ou tesouro encontrado durante a caça ao tesouro.
- **Peso (kg)**: O peso de cada item, expresso em quilogramas.
- **Valor ($)**: O valor monetário de cada item, expresso em dólares.
- **Descrição**: Uma breve descrição sobre o item, explicando sua importância ou raridade.

Aqui está o dataset que será utilizado:

| Item               | Peso (kg) | Valor ($) | Descrição                                                           |
|--------------------|-----------|-----------|---------------------------------------------------------------------|
| Moeda Antiga       | 1         | 300       | Moeda de ouro rara, valiosa para colecionadores.                    |
| Diamante           | 2         | 1500      | Pedra preciosa encontrada em um baú misterioso.                     |
| Barra de Ouro      | 5         | 2500      | Barra de ouro puro, mas muito pesada.                               |
| Colar de Prata     | 1.5       | 800       | Colar de prata decorado com pedras preciosas.                       |
| Poção Mágica       | 3         | 1200      | Poção mágica de valor inestimável para alquimistas.                 |
| Livro Antigo       | 2.5       | 500       | Livro antigo contendo segredos perdidos da civilização.             |
| Coroa              | 4         | 2200      | Coroa real incrustada com rubis e safiras.                          |
| Estátua de Jade    | 6         | 2800      | Estatueta de jade sagrado, venerada por povos antigos.              |
| Anel de Safira     | 0.5       | 900       | Anel de safira que pertencia a um rei lendário.                     |
| Mapa do Tesouro    | 1         | 1100      | Mapa que leva a um tesouro escondido, valioso para caçadores.       |


In [ ]:
# Instalando as dependências
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin"],
    license_uuid="default",
)

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


### Explicação do Modelo

O objetivo deste modelo é maximizar o valor total dos itens que um personagem pode carregar na mochila, respeitando o limite de peso. A função objetivo é dada por:

$$
\text{Maximizar valor_total} = \sum_{i \in \text{itens}} \text{valor}[i] \times \text{incluir}[i]
$$

Onde:
- **valor[i]** é o valor monetário do item \(i\).
- **incluir[i]** é uma variável binária que indica se o item \(i\) será incluído na mochila (1) ou não (0).

As restrições garantem que o peso total dos itens selecionados não ultrapasse a capacidade máxima da mochila:

$$
\sum_{i \in \text{itens}} \text{peso}[i] \times \text{incluir}[i] \leq \text{peso_max}
$$

Onde:
- **peso[i]** é o peso do item \(i\).
- **peso_max** é o peso máximo que a mochila pode carregar.

Este modelo é uma versão clássica do problema da mochila, onde buscamos otimizar a escolha de itens para maximizar o valor total sem exceder o limite de peso que é de 10 KG.


In [ ]:
%%writefile mochila.mod
# Definição do conjunto de itens
set ITEM;

# Parâmetros dos itens
param peso{ITEM};     # Peso de cada item (em kg)
param valor{ITEM};    # Valor de cada item (em dólares)
param peso_max;       # Peso máximo permitido na mochila

# Variáveis de decisão
var Incluir{j in ITEM} binary;  # 1 se o item for incluído, 0 caso contrário

# Função objetivo: maximizar o valor total dos itens na mochila
maximize Valor_Total: sum{j in ITEM} valor[j] * Incluir[j];

# Restrição de peso: o peso total dos itens selecionados não pode exceder o limite máximo da mochila
subject to Peso_Total: sum{j in ITEM} peso[j] * Incluir[j] <= peso_max;


Writing mochila.mod


In [ ]:
%%writefile mochila.dat
# Definição do conjunto de itens
set ITEM := Moeda_Antiga Diamante Barra_de_Ouro Colar_de_Prata Pocao_Magica Livro_Antigo Coroa Estatua_de_Jade Anel_de_Safira Mapa_do_Tesouro;

# Parâmetros dos itens
param peso :=
Moeda_Antiga      1
Diamante          2
Barra_de_Ouro     5
Colar_de_Prata    1.5
Pocao_Magica      3
Livro_Antigo      2.5
Coroa             4
Estatua_de_Jade   6
Anel_de_Safira    0.5
Mapa_do_Tesouro   1;

param valor :=
Moeda_Antiga      300
Diamante          1500
Barra_de_Ouro     2500
Colar_de_Prata    800
Pocao_Magica      1200
Livro_Antigo      500
Coroa             2200
Estatua_de_Jade   2800
Anel_de_Safira    900
Mapa_do_Tesouro   1100;

# Peso máximo da mochila
param peso_max := 10;


Writing mochila.dat


In [ ]:
%%ampl_eval
reset;
model mochila.mod;
data mochila.dat;
option solver cbc;
solve;
display Incluir;


cbc 2.10.12: cbc 2.10.12: optimal solution; objective 6800
0 simplex iterations
Incluir [*] :=
 Anel_de_Safira  1
  Barra_de_Ouro  0
 Colar_de_Prata  1
          Coroa  1
       Diamante  1
Estatua_de_Jade  0
   Livro_Antigo  0
Mapa_do_Tesouro  1
   Moeda_Antiga  1
   Pocao_Magica  0
;



## Análise do Resultado

O modelo foi resolvido com sucesso e a solução ótima foi encontrada. O valor total da mochila é **$6800**, o que significa que, ao selecionar os itens de acordo com a solução otimizada, o valor total dos itens na mochila atingiu 6800 dólares.

A solução indica quais itens foram incluídos na mochila (valor igual a **1**) e quais não foram (valor igual a **0**). A análise dos itens incluídos na mochila é a seguinte:

### Itens Selecionados:
1. **Anel de Safira**
2. **Colar de Prata**
3. **Coroa**
4. **Diamante**
5. **Mapa do Tesouro**
6. **Moeda Antiga**

### Itens Não Selecionados:
1. **Barra de Ouro**: Apesar de seu valor alto (2500), o peso (5 kg) pode ter sido um fator limitante.
2. **Estátua de Jade**: Seu valor é 2800, mas tem um peso de 6 kg, o que provavelmente excederia o limite de peso da mochila.
3. **Livro Antigo**: Seu valor é 500 e peso de 2.5 kg, mas a solução preferiu incluir outros itens de maior valor.
4. **Poção Mágica**: Apesar de seu valor de 1200, seu peso (3 kg) pode ter sido desvantajoso quando comparado aos outros itens selecionados.

### Peso Total:
A solução seleciona itens cujo peso total é \( 0.5 + 1.5 + 4 + 2 + 1 = 9 \) kg, que está abaixo do limite de peso máximo da mochila de **10 kg**.

### **Conclusão**:
A solução otimizada conseguiu maximizar o valor da mochila selecionando itens de alto valor e com pesos que se ajustam ao limite de 10 kg. A escolha de itens como o **Diamante**, a **Coroa** e o **Mapa do Tesouro** contribuiu significativamente para o valor total da solução. Além disso, a **Estátua de Jade** e a **Barra de Ouro** foram deixadas de fora devido ao seu alto peso em relação ao valor, o que demonstra a eficiência do modelo na escolha dos itens mais vantajosos.
